In [ ]:
# IMPORTS
from __future__ import print_function
from obspy import read
import sys
import obspy.clients.fdsn
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from obspy import read_events
from obspy import read, UTCDateTime as UTC
from obspy.signal.cross_correlation import correlation_detector
from obspy.signal.cross_correlation import correlate
import logging
errlog = logging.getLogger('obspy.warnings')
errlog.addHandler(logging.NullHandler())
import numpy as np
import obspy
from obspy.signal.cross_correlation import xcorr_pick_correction
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import pandas as pd
from scipy.fft import fft, fftfreq

In [ ]:
# OPTIONS  (IMPORTANT!)
#MAKE PARAMETERS DICT
# 1. DATA TYPE (REQUIRED) (EVENT OR AMBIENT NOISE)
corr_on = True
PARAMETERS = {}
PARAMETERS['DataType'] = 'noise'
# PARAMETERS['DataType'] = 'events'
#
# 2. TIME WINDOW (REQUIRED)
startdate_str = '2008-12-08'
data_time_len_days = 1/24
tlen_min = 1
tlen_sec = tlen_min*60
# tlen_min = 60
#
# 3. ARRAY PARAMETERS (REQUIRED)
array_params = {}
array_params['minlat'] = 38.7541 #only use stations within this box
array_params['maxlat'] = 41.6248
array_params['minlon'] = -106
array_params['maxlon'] = -102.8983
array_params['network'] = "TA" #only use these networks
array_params['starttime'] = UTCDateTime("2008-12-07")
array_params['endtime'] = UTCDateTime("2010-05-03")
array_params['fs'] = 50 #resample all traces to this frequency (Hertz)
array_params['tlen'] = tlen_sec
#
# 4. EVENT PARAMETERS (ONLY REQUIRED IF SET TO EVENT DATA TYPE [#1 above])
if PARAMETERS['DataType']=='events':
    events_params = {}
    events_params['minlat'] = 26 #only use events within this box
    events_params['maxlat'] = 49
    events_params['minlon'] = -125
    events_params['maxlon'] = -90
    events_params['minmag'] = 5.5 #only events with this magnitude or greater
    events_params['events_start'] = array_params['starttime'] #only events within this time window
    events_params['events_end'] = array_params['endtime']

In [ ]:
# PARAMETERS SUMMARY PRINTOUT SECTION
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------
# ---- don't touch this
# ---- don't touch this
starttime = '00:00:00'
start_date_time = startdate_str + 'T' + starttime
start_date = UTCDateTime(start_date_time)
end_date = start_date + data_time_len_days*24*60*60
traces_per_station = 0
current_start = start_date
current_end = start_date + tlen_sec
while current_end<(end_date+1):
    traces_per_station+=1
    current_end = current_start + tlen_sec
    # print('# ' + str(num) + ': ' + 'Start: ' + str(current_start) + ' End: ' + str(current_end))
    current_start = current_end
# ---- don't touch this
# ---- don't touch this
# ----
print('-------S T A T I O N    C A T A L O G-------')
print('-------O24A IS LOCATED IN BOULDER-------')
# DO NOT TOUCH ANY OF THIS
PARAMETERS['include_traces'] = False
Features = ['SRC_ID','REC_ID','SRC_Lat','SRC_Lon','REC_Lat','REC_Lon','Greens']
if PARAMETERS['include_traces']:
    Features.append('SRC_Trace')
    Features.append('REC_Trace')
PARAMETERS['Features'] = Features
PARAMETERS['Dataset_Length_Days'] = data_time_len_days
PARAMETERS['StartDate'] = start_date
PARAMETERS['EndDate'] = end_date
PARAMETERS['Trace_Length_Seconds'] = tlen_sec
PARAMETERS['Array_Params'] = array_params
if PARAMETERS['DataType']=='events':
    PARAMETERS['Event_Data_Options'] = events_params
array_inventory = [0]
try:
    client = Client("IRIS")
    array_inventory = client.get_stations(network=PARAMETERS['Array_Params']['network'], minlatitude=PARAMETERS['Array_Params']['minlat'],maxlatitude=PARAMETERS['Array_Params']['maxlat'],minlongitude=PARAMETERS['Array_Params']['minlon'],maxlongitude=PARAMETERS['Array_Params']['maxlon'],endtime=PARAMETERS['Array_Params']['endtime'],starttime=PARAMETERS['Array_Params']['starttime'])
    array_inventory.plot(projection="local",color_per_network=True,show=False)
    PARAMETERS['Array_Params']['inventory'] = array_inventory
    plt.show()
except Exception:
    print('FAILURE: TIMEOUT CONNECTION ISSUE')
    sys.exit()
try:
    if PARAMETERS['DataType']=='events':
        print('-------E V E N T    C A T A L O G-------')
        event_catalog = client.get_events(starttime=PARAMETERS['Event_Data_Options']['events_start'], endtime=PARAMETERS['Event_Data_Options']['events_end'],minmagnitude=PARAMETERS['Event_Data_Options']['minmag'],minlatitude=PARAMETERS['Event_Data_Options']['minlat'],maxlatitude=PARAMETERS['Event_Data_Options']['maxlat'],minlongitude=PARAMETERS['Event_Data_Options']['minlon'],maxlongitude=PARAMETERS['Event_Data_Options']['maxlon'])
        event_catalog.plot(projection='local')
        PARAMETERS['Event_Data_Options']['catalog'] = event_catalog
        PARAMETERS['Event_Data_Options']['catalog']
except Exception:
    print('FAILURE: TIMEOUT CONNECTION ISSUE')
    sys.exit()
num_stations = len(PARAMETERS['Array_Params']['inventory'][0]) #can collect multiple networks, but I have hardcoded here to look at only one network (index=0), the TA network.
num_source_receiver_pairs_per_sta = (num_stations**2)-num_stations
# ----
PARAMETERS['Number_of_Stations'] = num_stations
PARAMETERS['Traces_Per_Station'] = traces_per_station
PARAMETERS['Greens_per_Station_per_Trace'] = num_source_receiver_pairs_per_sta - 1
PARAMETERS['DataSet_Size'] = (num_source_receiver_pairs_per_sta-1)*traces_per_station
# maybe I will need Boulder Lat/Lon at some point...
PARAMETERS['Boulder_LaLo'] = [40.0150,-105.2705]

print('-')
print('-')
print('-')
print('---PARAMETERS SUMMARY---')
print('Data Type: ' + PARAMETERS['DataType'].replace('events','Earthquake Events').replace('noise','Ambient Noise'))
print('Data Samplerate (Hertz): ' + str(array_params['fs']) + ' <- This parameter tunes spectral resolution')
print('--TRACE SEGMENT LENGTHS (SECONDS): ' + str(PARAMETERS['Trace_Length_Seconds']) + ' <- This parameter tunes signal resolition primarily. If it is too small it will also affect spectral resolution.')
print('--DATA SET TIME LENGTH (DAYS): ' + str(PARAMETERS['Dataset_Length_Days']) + ' <- This parameter tunes signal resolution')
print('--NUMBER OF STATIONS: ' + str(PARAMETERS['Number_of_Stations']) + ' <- This parameter tunes spatial resolution')
print('-')
print('--METRIC TOTALS--')
print('--GREENS FOR EACH RAY PATH: ' + str(PARAMETERS['Traces_Per_Station']-1))
print('--GREENS PER STATION-TRACE: ' + str(PARAMETERS['Greens_per_Station_per_Trace']))
print('-')
print('--!!IMPORTANT!!')
print('-TOTAL SIZE OF DATA SET:: ' + str(PARAMETERS['DataSet_Size']) + ' <- This is directly proportional to the bandwidth and computational demand for downloading the data set and training the neural network.')
print('-K_FOLD: ' + str(PARAMETERS['Traces_Per_Station']-1) + ' <- This is the signal stack size for each ray path. This is the most direct measurement of projected signal quality.')
print('-Predicted SNR: ' + str((PARAMETERS['Traces_Per_Station']**0.5)) + ' <- A rough estimation of the signal quality we should expect. Anything over 50 is fairly descent. 45 is the bare minimum we would want.')
print('-')
print('-- UTC DATE FORMAT: YYYY-MM-DD T HH:MM:SS')
print(' - --DATA SET START DATE:')
print(PARAMETERS['StartDate'])
print(' - --DATA SET END DATE:')
print(PARAMETERS['EndDate'])
print('-------------------')
print('-')
print('-')
print('-')
print('---ALL PARAMETERS (VERBOSE)----')
PARAMETERS
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------

In [ ]:
def slice(trace_obj,tlen):
    COMPONENT = ['Z','N','E']
    out = {}
    out[COMPONENT[0]] = []
    out[COMPONENT[1]] = []
    out[COMPONENT[2]] = []
    for comp_k in range(len(COMPONENT)):
        for win in zip(trace_obj.slide(window_length=tlen,step=tlen,include_partial_windows=False,nearest_sample=True)):
            out[COMPONENT[comp_k]].append(list(win[0].select(component=COMPONENT[comp_k])[0]))
    return out
# -------------------------------------------------------------
def Greens_Fourier(SRC,REC):
    G = {}
    COMPONENT = list(SRC.keys())
    Nslices = len(SRC[list(SRC.keys())[0]])
    G[COMPONENT[0]] = []
    G[COMPONENT[1]] = []
    G[COMPONENT[2]] = []
    for win_k in range(Nslices):
        for comp_k in range(len(COMPONENT)):
            A = SRC[COMPONENT[comp_k]][win_k]
            B = REC[COMPONENT[comp_k]][win_k]
            Lshift = len(A)
            cc = sc.signal.correlate(A, B, mode='full', method='fft')
            if len(cc)!=Lshift:
                cc = cc[0:Lshift]
            G[COMPONENT[comp_k]].append(cc)
    return G
# -------------------------------------------------------------
def Greens_Time(SRC,REC):
    G = {}
    COMPONENT = list(SRC.keys())
    Nslices = len(SRC[list(SRC.keys())[0]])
    G[COMPONENT[0]] = []
    G[COMPONENT[1]] = []
    G[COMPONENT[2]] = []
    for win_k in range(Nslices):
        for comp_k in range(len(COMPONENT)):
            A = SRC[COMPONENT[comp_k]][win_k]
            B = REC[COMPONENT[comp_k]][win_k]
            Lshift = len(A)
            cc = correlate(A, B, Lshift)
            cc = cc[0:Lshift]
            cc = np.flip(cc)
            cc = sc.signal.detrend(cc)
            G[COMPONENT[comp_k]].append(cc)
    return G
# -------------------------------------------------------------
def rotate(TRC,ba,inc):
    OUT = {}
    OUT['L'] = []
    OUT['Q'] = []
    OUT['T'] = []
    M3D = np.array(([np.cos(inc),-np.sin(inc)*np.sin(ba),-np.sin(inc)*np.cos(ba)],
    [np.sin(inc),np.cos(inc)*np.sin(ba),np.cos(inc)*np.cos(ba)],
    [0,-np.cos(ba),np.sin(ba)]))
    for win_k in range(len(TRC['Z'])):
        ZEN = np.array((TRC['Z'][win_k],TRC['E'][win_k],TRC['N'][win_k])) # <--ZEN is NOT a typo..
        LQT = np.matmul(M3D,ZEN)
        OUT['L'].append(sc.signal.detrend(LQT[0,:]))
        OUT['Q'].append(sc.signal.detrend(LQT[1,:]))
        OUT['T'].append(sc.signal.detrend(LQT[2,:]))
    return OUT
# -------------------------------------------------------------


In [ ]:
# rec_trace_object = client.get_waveforms(rec_net, rec_sta, "*", "BH?", t1, t2)

In [ ]:
if PARAMETERS['DataType']=='noise':
    # pre-allocate dataset dataframe
    tlen = PARAMETERS['Trace_Length_Seconds']
    COMPONENT = 'L'
    DATA = pd.DataFrame(columns=PARAMETERS['Features'])
    Failures = 0
    Successes = 0
    # plot_on = True
    plot_on = False
    rotate_on = True
    corr_on = True
    num = 0
    Ntimesteps = 0
    current_start = PARAMETERS['StartDate']
    current_end = 0
    chunks = 60*60
    while current_end<(PARAMETERS['EndDate']):
        Ntimesteps+=1
        current_end = current_start + chunks
        current_start = current_end
    current_start = PARAMETERS['StartDate']
    current_end = 0
    while current_end<(PARAMETERS['EndDate']):
        srpair = 0
        num+=1
        time_pct = np.round((100*num/Ntimesteps)*1000)/1000
        chunks = 60*60
        # current_end = current_start + PARAMETERS['Trace_Length_Seconds']
        current_end = current_start + chunks
        print('# ' + str(num) + ': ' + 'Start: ' + str(current_start) + ' End: ' + str(current_end))
        tcut = 5
        t1 = current_start
        t2 = current_end + tcut
        net_k = 0
        N = PARAMETERS['Number_of_Stations']
        # N = 3
        for src_ind in range(N):
            for rec_ind in range(N):
                successful_steps = 0
                # Data Collect Step ONE: Define the Query Meta
                if src_ind==rec_ind:
                    continue
                else:
                    srpair+=1
                    sr_pct = np.round((100*srpair/((N*N)-N))*1000)/1000
                    pair_string = 'Time Step: ' + str(time_pct) + '%' +  ' (' + str(num) + '/' + str(Ntimesteps) + ') | Source-Receiver Pair: ' + str(sr_pct) + '%' + ' (' + str(srpair) + '/' + str((N*N)-N) + ')'
                    # print('SOURCE-RECEIVER PAIR #' +str(srpair))
                    src_net = PARAMETERS['Array_Params']['inventory'][net_k].code
                    src_sta = PARAMETERS['Array_Params']['inventory'][net_k][src_ind].code
                    src_x = PARAMETERS['Array_Params']['inventory'][net_k][src_ind].longitude
                    src_y = PARAMETERS['Array_Params']['inventory'][net_k][src_ind].latitude

                    rec_net = PARAMETERS['Array_Params']['inventory'][net_k].code
                    rec_sta = PARAMETERS['Array_Params']['inventory'][net_k][rec_ind].code
                    rec_x = PARAMETERS['Array_Params']['inventory'][net_k][rec_ind].longitude
                    rec_y = PARAMETERS['Array_Params']['inventory'][net_k][rec_ind].latitude

                    # Data Collect Step TWO: Download the data
                    try:
                        src_trace_object = client.get_waveforms(src_net, src_sta, "*", "BH?", t1, t2)
                        successful_steps+=1
                    except Exception:
                        print(pair_string + ' | Result: FAILURE (TIMEOUT/CONNECTION ISSUE)')
                        Failures += 1
                        continue
                    try:
                        rec_trace_object = client.get_waveforms(rec_net, rec_sta, "*", "BH?", t1, t2)
                        successful_steps+=1
                    except Exception:
                        print(pair_string + ' | Result: FAILURE (TIMEOUT/CONNECTION ISSUE)')
                        Failures += 1
                        Data_Retrieved = 0
                        continue
                    if successful_steps==2:
                        try:
                            # Data Collect Step THREE: PREPROCESSING-RESAMPLE
                            src_trace_object.detrend(type='linear')
                            src_trace_object = src_trace_object.resample(PARAMETERS['Array_Params']['fs'])
                            src_trace_object = src_trace_object.trim(current_start,current_end,nearest_sample=True,pad=True,fill_value=0)
                            src_trace_object.normalize(global_max=True)
                            successful_steps+=1
                            rec_trace_object = rec_trace_object.detrend(type='linear')
                            rec_trace_object = rec_trace_object.resample(PARAMETERS['Array_Params']['fs'])
                            rec_trace_object = rec_trace_object.trim(current_start,current_end,nearest_sample=True,pad=True,fill_value=0)
                            rec_trace_object.normalize(global_max=True)
                            successful_steps+=1
                        except:
                            print(pair_string + ' | Result: FAILURE (DATA PROCESSING ISSUE)')
                            Failures += 1
                            continue
                    S = slice(src_trace_object,tlen)
                    R = slice(rec_trace_object,tlen)
                    # print('Successful steps: ' + str(successful_steps))
                    # NON-ROTATED 3-C Green's
                    G_ZNE = Greens_Fourier(S,R)
                    # Data Collect Step FOUR: PREPROCESSING-ROTATION
                    if successful_steps==4:
                        if rotate_on:
                            source_lla = [src_y,src_x]
                            receiver_lla = [rec_y,rec_x]
                            dst,az,ba = obspy.geodetics.base.gps2dist_azimuth(float(source_lla[0]),float(source_lla[1]),float(receiver_lla[0]),float(receiver_lla[1]))
                            dz = float(0)
                            dx = dst
                            if dz==float(0):
                                inc = 90
                            else:
                                inc = np.arctan(dx/dz)
                            G_LQT = rotate(G_ZNE,ba,inc)
                            successful_steps+=2
                            # print('Rotation Complete: ' + str(successful_steps))
                        # Data Collect Step FIVE: PREPROCESSING-PLOTTING (OPTIONAL)
                    if successful_steps==6:
                        if plot_on:
                            print('-Source Trace-')
                            src_trace_object.plot()
                            # src_trace_object[0].spectrogram(log=True,title='P-wave Spectrogram (L)')
                            # src_trace_object[1].spectrogram(log=True,title='SV-wave Spectrogram (Q)')
                            # src_trace_object[2].spectrogram(log=True,title='SH-wave Spectrogram (T)')
                            plt.show()
                            print('-Receiver Trace-')
                            rec_trace_object.plot()
                            # rec_trace_object[0].spectrogram(log=True,title='P-wave Spectrogram (L)')
                            # rec_trace_object[1].spectrogram(log=True,title='SV-wave Spectrogram (Q)')
                            # rec_trace_object[2].spectrogram(log=True,title='SH-wave Spectrogram (T)')
                            plt.show()
                    if successful_steps<6:
                        print(pair_string + ' | Result: FAILURE (ROTATION ISSUE) : ' + str(successful_steps))
                        Failures += 1
                        continue
                    else:
                        if corr_on:
                            tlen = PARAMETERS['Trace_Length_Seconds']
                            # tlen = 5
                            Strace = src_trace_object
                            Rtrace = rec_trace_object
                            Nslices = len(G_LQT['L'])
                            for k_ind in range(Nslices):
                                G_k = G_LQT['L'][k_ind]
                                if PARAMETERS['include_traces']:
                                    DATA = DATA.append(pd.DataFrame(data=[[src_ind,rec_ind,src_y,src_x,rec_y,rec_x,G_k,Strace,Rtrace]],columns=PARAMETERS['Features']))
                                else:
                                    DATA = DATA.append(pd.DataFrame(data=[[src_ind,rec_ind,src_y,src_x,rec_y,rec_x,G_k]],columns=PARAMETERS['Features']))
                            print(pair_string + ' | Result: SUCCESS')
                            Successes += 1
                        else:
                            Successes += 1
                    # if srpair==2:
                        # q = kqwe
        current_start = current_end
DATA
print('COMPLETE')

In [ ]:

# import time
# for ind in range(20):
#     Fs = 50
#     Y = DATA.iloc[ind].Greens.copy()
#     Y = Y/np.max(Y)
#     X = np.arange(0,len(Y),1)/Fs
#     plt.scatter(X,Y,color=plt.cm.gist_ncar(Y))
#     plt.title('RMS=' + str(np.mean(Y**2)**0.5))
#     plt.show()
#     time.sleep(1.5)


In [24]:
def plot_traces(D,i=0,fs=1):
    GF = D.iloc[i].Greens
    time = np.arange(0,len(GF),1)/fs
    plt.figure(figsize=[10,10])
    if 'SRC_Trace' in D.columns:
        if 'REC_Trace' in D.columns:
            S_TRC = D.iloc[i].REC_Trace
            R_TRC = D.iloc[i].SRC_Trace
            plt.subplot(212)
            plt.plot(time,S_TRC,c='r',label='Source')
            plt.plot(time,R_TRC,c='b',label='Receiver')
            plt.xlim(np.min(time),np.max(time))
            plt.xlabel('time (s)')
            plt.legend()
            plt.subplot(211)
    plt.plot(time,GF,'g',label='Greens')
    plt.xlim(np.min(time),np.max(time))
    plt.xlabel('time (s)')
    plt.ylabel('Coeff',fontweight='bold',fontsize=15)
    plt.legend()
#
def stack_greens(D,hilbert=False):
    F = ['SRC_ID','REC_ID','SRC_Lat','SRC_Lon','REC_Lat','REC_Lon','Greens']
    STACK = pd.DataFrame(columns=F)
    D = D[F]
    N = len(np.max((D.SRC_ID.unique().tolist(),D.REC_ID.unique().tolist())))
    for sid in range(N):
        for rid in range(N):
            if rid==sid:
                continue
            DD = D.loc[(D['SRC_ID'] == sid) & (D['REC_ID'] == rid)].copy()
            cstack = DD.mean(axis=0).to_frame().transpose()
            # if hilbert:
            # cstack.Greens = np.real(sc.signal.hilbert(cstack.Greens.tolist()).tolist()).tolist()
            STACK = STACK.append(cstack)
    return STACK
#
def map_greens(D,sid=0):
    DS = D.loc[(D['SRC_ID'] == sid)].copy()
    XY = D[['SRC_Lat','SRC_Lon']].copy().drop_duplicates()
    X = XY.SRC_Lon.to_list()
    Y = XY.SRC_Lat.to_list()
    plt.scatter(X,Y,marker='^',c='r',s=150)
    N = len(DS)
    # N = 1
    for ii in range(N):
        sy = DS.iloc[ii].SRC_Lat
        sx = DS.iloc[ii].SRC_Lon
        ry = DS.iloc[ii].REC_Lat
        rx = DS.iloc[ii].REC_Lon
        gf = DS.iloc[ii].Greens
        xlon = np.linspace(sx,rx,len(gf))
        xlat = np.linspace(sy,ry,len(gf))
        plt.scatter(xlon,xlat,c=plt.cm.gist_ncar(gf))
    plt.xlabel('LONGITUDE',fontweight='bold',fontsize=15)
    plt.ylabel('LATITUDE',fontweight='bold',fontsize=15)
    plt.xticks(fontweight='bold',fontsize=15)
    plt.yticks(fontweight='bold',fontsize=15)

In [ ]:
# if PARAMETERS['DataType']=='events':
#     receiver_inventory = []
#     receiver_trace = []
#     event_k = 6 #33=m6.9
#     sta_k = 19 #0=M23A #19=Q26A
#     net_k = 0 #0=TA
#     for sta_k in range(len(PARAMETERS['Array_Params']['inventory'][net_k])):
#         event_origin = PARAMETERS['Event_Data_Options']['catalog'][event_k].origins[0].time
#         sta = PARAMETERS['Array_Params']['inventory'][net_k][sta_k].code
#         net = PARAMETERS['Array_Params']['inventory'][net_k].code
#         print(event_origin)
#         print(net,sta)
#         t1 = event_origin
#         t2 = event_origin + PARAMETERS['Array_Params']['tlen']
#         st = client.get_waveforms(net, sta, "*", "BH?", t1, t2)
#         st.resample(PARAMETERS['Array_Params']['fs'])
#         source_lla = [PARAMETERS['Event_Data_Options']['catalog'][event_k].origins[0].latitude,PARAMETERS['Event_Data_Options']['catalog'][event_k].origins[0].longitude]
#         receiver_lla = [PARAMETERS['Array_Params']['inventory'][net_k][sta_k].latitude,PARAMETERS['Array_Params']['inventory'][net_k][sta_k].longitude]
#         rotate_on = True
#         if rotate_on:
#             dst,az,ba = obspy.geodetics.base.gps2dist_azimuth(float(source_lla[0]),float(source_lla[1]),float(receiver_lla[0]),float(receiver_lla[1]))
#             dz = float(0)
#             dx = dst
#             if dz==float(0):
#                 inc = 90
#             else:
#                 inc = np.arctan(dx/dz)
#             method = 'ZNE->LQT'
#             # L,Q,T = obspy.signal.rotate.rotate_zne_lqt(z,n,e,ba,inc)
#             st.rotate(method,ba,inc,st)
#         receiver_inventory.append(PARAMETERS['Array_Params']['inventory'][net_k][sta_k])
#         receiver_trace.append(st)
#     st.plot()
#     st[0].spectrogram(log=True,title='P-wave Spectrogram (L)')
#     st[1].spectrogram(log=True,title='SV-wave Spectrogram (Q)')
#     st[2].spectrogram(log=True,title='SH-wave Spectrogram (T)')
#     events_params['catalog'][event_k]